In [ ]:
import argparse
import warnings
warnings.filterwarnings("ignore")

def main (args):  

  # Load data
  directory = "/content/drive/MyDrive/CS230_Project/ControlData/Trials/"
  X_train, Y_train, X_test, Y_test = data_initialize(directory)
  
  # Model traininig / testing
  model_parameters = {'task': args.task,
                      'model_type': args.model_type,
                      'h_dim': args.h_dim,
                      'n_layer': args.n_layer,
                      'batch_size': args.batch_size,
                      'epoch': args.epoch,
                      'learning_rate': args.learning_rate}
  
  if args.model_type in ['rnn','lstm','gru']:
    general_rnn = GeneralRNN(model_parameters)    
    out = general_rnn.fit(X_train, Y_train, X_test, Y_test)
  
  # Evaluation
  loss_graph(out, args.epoch)
  acc_graph(out, args.epoch)
  misc_graph(X1) 
    
##  
if __name__ == '__main__':
  args = args()
  # Call main function  
  main(args)